# État de la malnutrition dans le monde

## Partie 1 - Inspection des données

Je commence par importer les librairies pandas et numpy.

In [637]:
import pandas as pd
import numpy as np

Lecture et analyse du premier fichier csv.

In [638]:
aide_alimentaire = pd.read_csv('aide_alimentaire.csv')
aide_alimentaire.head()

,Pays bénéficiaire,Année,Produit,Valeur
0,Afghanistan,2013,Autres non-céréales,682
1,Afghanistan,2014,Autres non-céréales,335
2,Afghanistan,2013,Blé et Farin,39224
3,Afghanistan,2014,Blé et Farin,15160
4,Afghanistan,2013,Céréales,40504


In [639]:
aide_alimentaire = aide_alimentaire.rename(columns={"Pays bénéficiaire": "Pays", "Valeur": "Quantité (t)"})
aide_alimentaire.head()

,Pays,Année,Produit,Quantité (t)
0,Afghanistan,2013,Autres non-céréales,682
1,Afghanistan,2014,Autres non-céréales,335
2,Afghanistan,2013,Blé et Farin,39224
3,Afghanistan,2014,Blé et Farin,15160
4,Afghanistan,2013,Céréales,40504


In [640]:
aide_alimentaire.shape

(1475, 4)

In [641]:
aide_alimentaire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1475 entries, 0 to 1474
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Pays          1475 non-null   object
 1   Année         1475 non-null   int64 
 2   Produit       1475 non-null   object
 3   Quantité (t)  1475 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 46.2+ KB


Le permier est en ordre je peux passer au second fichier csv.

In [642]:
population = pd.read_csv('population.csv')
population.head()

,Zone,Année,Valeur
0,Afghanistan,2013,32269.589
1,Afghanistan,2014,33370.794
2,Afghanistan,2015,34413.603
3,Afghanistan,2016,35383.032
4,Afghanistan,2017,36296.113


Ci dessous je multiplie la colonne Ptot par 1000 pour obtenir des millions

In [643]:
population = population.rename(columns={"Zone": "Pays", 'Valeur': 'Ptot (M)'})
population['Ptot (M)'] = population['Ptot (M)'] * 1000
population.head()

,Pays,Année,Ptot (M)
0,Afghanistan,2013,32269589.0
1,Afghanistan,2014,33370794.0
2,Afghanistan,2015,34413603.0
3,Afghanistan,2016,35383032.0
4,Afghanistan,2017,36296113.0


In [645]:
population.shape

(1416, 3)

In [646]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1416 entries, 0 to 1415
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pays      1416 non-null   object 
 1   Année     1416 non-null   int64  
 2   Ptot (M)  1416 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 33.3+ KB


De nouveau le fichier sur la population est propre je commence donc l'analyse du troisieme fichier.

In [648]:
sous_nutrition = pd.read_csv('sous_nutrition.csv')
sous_nutrition.head()

,Zone,Année,Valeur
0,Afghanistan,2012-2014,8.6
1,Afghanistan,2013-2015,8.8
2,Afghanistan,2014-2016,8.9
3,Afghanistan,2015-2017,9.7
4,Afghanistan,2016-2018,10.5


In [649]:
sous_nutrition = sous_nutrition.rename(columns={"Zone": "Pays", 'Valeur': 'Personnes en sous alimentation (M)'})
sous_nutrition.head()

,Pays,Année,Personnes en sous alimentation (M)
0,Afghanistan,2012-2014,8.6
1,Afghanistan,2013-2015,8.8
2,Afghanistan,2014-2016,8.9
3,Afghanistan,2015-2017,9.7
4,Afghanistan,2016-2018,10.5


In [650]:
sous_nutrition.dtypes

Pays                                  object
Année                                 object
Personnes en sous alimentation (M)    object
dtype: object

Ici je ne pouvais pas changer la colonne 2 en float à cause de la présence du caractère "<", j'ai donc remplacer ce caractère pour pouvoir mettre en float. Et j'ai decidé de passer la valeur <0.1 en 0, pour ne pas fausser mes résultats.

In [651]:
sous_nutrition['Personnes en sous alimentation (M)'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]',
value=r'')

In [652]:
sous_nutrition['Personnes en sous alimentation (M)'].replace('0.1', '0', inplace=True)
sous_nutrition.head(62)

,Pays,Année,Personnes en sous alimentation (M)
0,Afghanistan,2012-2014,8.6
1,Afghanistan,2013-2015,8.8
2,Afghanistan,2014-2016,8.9
3,Afghanistan,2015-2017,9.7
4,Afghanistan,2016-2018,10.5
...,...,...,...
57,Argentine,2015-2017,1.3
58,Argentine,2016-2018,1.5
59,Argentine,2017-2019,1.7
60,Arménie,2012-2014,0


In [818]:
sous_nutrition['Personnes en sous alimentation (M)'] = sous_nutrition['Personnes en sous alimentation (M)'].astype(float)
sous_nutrition.dtypes

Pays                                   object
Année                                   int64
Personnes en sous alimentation (M)    float64
dtype: object

Ci dessous je multiplie la colonne Personne en sous alimentation  par 1.000.000 pour obtenir des millions.

In [654]:
sous_nutrition['Personnes en sous alimentation (M)'] = sous_nutrition['Personnes en sous alimentation (M)'] * 1000000
sous_nutrition.head()

,Pays,Année,Personnes en sous alimentation (M)
0,Afghanistan,2012-2014,8600000.0
1,Afghanistan,2013-2015,8800000.0
2,Afghanistan,2014-2016,8900000.0
3,Afghanistan,2015-2017,9700000.0
4,Afghanistan,2016-2018,10500000.0


Maintenant je remplace les années sur "trois ans" par l'année du milieu.

In [656]:
sous_nutrition['Année'].replace(['2012-2014', '2013-2015', '2014-2016', '2015-2017', '2016-2018', '2017-2019'], ['2013', '2014', '2015', '2016', '2017', '2018'], inplace=True)
sous_nutrition.head()

,Pays,Année,Personnes en sous alimentation (M)
0,Afghanistan,2013,8600000.0
1,Afghanistan,2014,8800000.0
2,Afghanistan,2015,8900000.0
3,Afghanistan,2016,9700000.0
4,Afghanistan,2017,10500000.0


In [657]:
sous_nutrition['Année'] = sous_nutrition['Année'].astype(int)

In [658]:
sous_nutrition.shape

(1218, 3)

In [659]:
sous_nutrition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Pays                                1218 non-null   object 
 1   Année                               1218 non-null   int64  
 2   Personnes en sous alimentation (M)  624 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 28.7+ KB


Pour terminer sur l'analyse des fichiers csv je passe à celui sur la disponibilité alimentaire.

In [660]:
dispo_alimentaire = pd.read_csv('dispo_alimentaire.csv')
dispo_alimentaire.head()

,Zone,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,Afghanistan,Abats Comestible,animale,NaN,NaN,5.0,1.72,0.20,0.77,53.0,NaN,NaN,53.0,NaN,53.0,NaN,NaN,NaN
1,Afghanistan,"Agrumes, Autres",vegetale,NaN,NaN,1.0,1.29,0.01,0.02,41.0,2.0,40.0,39.0,2.0,3.0,NaN,NaN,NaN
2,Afghanistan,Aliments pour enfants,vegetale,NaN,NaN,1.0,0.06,0.01,0.03,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,Ananas,vegetale,NaN,NaN,0.0,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Bananes,vegetale,NaN,NaN,4.0,2.70,0.02,0.05,82.0,NaN,82.0,82.0,NaN,NaN,NaN,NaN,NaN


In [661]:
dispo_alimentaire = dispo_alimentaire.rename(columns={"Zone": "Pays"})
dispo_alimentaire.head()

,Pays,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,Afghanistan,Abats Comestible,animale,NaN,NaN,5.0,1.72,0.20,0.77,53.0,NaN,NaN,53.0,NaN,53.0,NaN,NaN,NaN
1,Afghanistan,"Agrumes, Autres",vegetale,NaN,NaN,1.0,1.29,0.01,0.02,41.0,2.0,40.0,39.0,2.0,3.0,NaN,NaN,NaN
2,Afghanistan,Aliments pour enfants,vegetale,NaN,NaN,1.0,0.06,0.01,0.03,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,Ananas,vegetale,NaN,NaN,0.0,0.00,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Bananes,vegetale,NaN,NaN,4.0,2.70,0.02,0.05,82.0,NaN,82.0,82.0,NaN,NaN,NaN,NaN,NaN


Je remplace toutes les valeurs "NaN" par 0.

In [824]:
dispo_alimentaire = dispo_alimentaire.fillna(0)
dispo_alimentaire.head()

,Pays,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,Afghanistan,Abats Comestible,animale,0.0,0.0,5.0,1.72,0.20,0.77,53.0,0.0,0.0,53.0,0.0,53.0,0.0,0.0,0.0
1,Afghanistan,"Agrumes, Autres",vegetale,0.0,0.0,1.0,1.29,0.01,0.02,41.0,2.0,40.0,39.0,2.0,3.0,0.0,0.0,0.0
2,Afghanistan,Aliments pour enfants,vegetale,0.0,0.0,1.0,0.06,0.01,0.03,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,Ananas,vegetale,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,Bananes,vegetale,0.0,0.0,4.0,2.70,0.02,0.05,82.0,0.0,82.0,82.0,0.0,0.0,0.0,0.0,0.0


In [822]:
dispo_alimentaire.shape

(15605, 18)

In [823]:
dispo_alimentaire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15605 entries, 0 to 15604
Data columns (total 18 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Pays                                                           15605 non-null  object 
 1   Produit                                                        15605 non-null  object 
 2   Origine                                                        15605 non-null  object 
 3   Aliments pour animaux                                          15605 non-null  float64
 4   Autres Utilisations                                            15605 non-null  float64
 5   Disponibilité alimentaire (Kcal/personne/jour)                 15605 non-null  float64
 6   Disponibilité alimentaire en quantité (kg/personne/an)         15605 non-null  float64
 7   Disponibilité de matière grasse en quantité (g/personne/jo

## Partie 2 - Questions de Marc

1. Proportion de personnes en état de sous-nutrition en 2017 dans le monde.

In [825]:
proportion = pd.merge (sous_nutrition, population, how='inner', on=['Pays', 'Année'])
proportion.head()

,Pays,Année,Personnes en sous alimentation (M),Ptot (M)
0,Afghanistan,2013,8600000.0,32269589.0
1,Afghanistan,2014,8800000.0,33370794.0
2,Afghanistan,2015,8900000.0,34413603.0
3,Afghanistan,2016,9700000.0,35383032.0
4,Afghanistan,2017,10500000.0,36296113.0


In [826]:
proportion = proportion.loc[proportion['Année'] == 2017, :]
proportion.head()

,Pays,Année,Personnes en sous alimentation (M),Ptot (M)
4,Afghanistan,2017,10500000.0,36296113.0
10,Afrique du Sud,2017,3100000.0,57009756.0
16,Albanie,2017,0.0,2884169.0
22,Algérie,2017,1300000.0,41389189.0
28,Allemagne,2017,NaN,82658409.0


In [829]:
proportion_sous_alimentation_2017 = round(((proportion['Personnes en sous alimentation (M)'].sum())/(proportion['Ptot (M)'].sum()))*100, 2)
print('En 2017, il y a', proportion_sous_alimentation_2017, '% de personne en sous-nutrition dans le monde.')

En 2017, il y a 7.1 % de personne en sous-nutrition dans le monde


2. Le nombre théorique de personnes qui pourraient être nourries. Tu devrais pouvoir calculer ça à partir de la disponibilité alimentaire mondiale.

In [830]:
population_2017 = population.loc[population['Année'] == 2017, :]
population_2017.head()

,Pays,Année,Ptot (M)
4,Afghanistan,2017,36296113.0
10,Afrique du Sud,2017,57009756.0
16,Albanie,2017,2884169.0
22,Algérie,2017,41389189.0
28,Allemagne,2017,82658409.0


Pour 2017, je regroupe la disponibilité alimentaire (par personne) par pays et j'y joins le population totale du pays.

In [834]:
pays_kcal = dispo_alimentaire.iloc[:, 0:6]
pays_kcal = pays_kcal.groupby('Pays').agg(np.sum)
pays_kcal = pd.merge(pays_kcal, population_2017, how='inner', left_on='Pays', right_on='Pays')
pays_kcal[['Pays', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Année', 'Ptot (M)']]


,Pays,Disponibilité alimentaire (Kcal/personne/jour),Année,Ptot (M)
0,Afghanistan,2087.0,2017,36296113.0
1,Afrique du Sud,3020.0,2017,57009756.0
2,Albanie,3188.0,2017,2884169.0
3,Algérie,3293.0,2017,41389189.0
4,Allemagne,3503.0,2017,82658409.0
...,...,...,...,...
167,Émirats arabes unis,3275.0,2017,9487203.0
168,Équateur,2346.0,2017,16785361.0
169,États-Unis d'Amérique,3682.0,2017,325084756.0
170,Éthiopie,2129.0,2017,106399924.0


Ensuite je crée une nouvelle colonne où j'obtiens la disponibilité alimentaire totale par pays pour tous les habitants.

In [838]:
pays_kcal['dispo_par_pays'] = pays_kcal['Disponibilité alimentaire (Kcal/personne/jour)'] * pays_kcal['Ptot (M)']
pays_kcal[['Pays', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Année', 'Ptot (M)', 'dispo_par_pays']]

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Année,Ptot (M),dispo_par_pays
0,Afghanistan,2087.0,2017,36296113.0,7.574999e+10
1,Afrique du Sud,3020.0,2017,57009756.0,1.721695e+11
2,Albanie,3188.0,2017,2884169.0,9.194731e+09
3,Algérie,3293.0,2017,41389189.0,1.362946e+11
4,Allemagne,3503.0,2017,82658409.0,2.895524e+11
...,...,...,...,...,...
167,Émirats arabes unis,3275.0,2017,9487203.0,3.107059e+10
168,Équateur,2346.0,2017,16785361.0,3.937846e+10
169,États-Unis d'Amérique,3682.0,2017,325084756.0,1.196962e+12
170,Éthiopie,2129.0,2017,106399924.0,2.265254e+11


Enfin j'aditionne toutes les disponibilité alimentaire mondiale d'une journée et divise ce total par 2400 kcal, car d'après ce site (https://www.vidal.fr/sante/nutrition/equilibre-alimentaire-adulte/recommandations-nutritionnelles-adulte.html) il s'agit d'un apport nutritionnel moyen recommandé.

In [840]:
nbre_theorique = round((pays_kcal['dispo_par_pays'].sum())/2400, 2)
print("En théorie, la disponibilité alimentaire mondiale d'une journée permettrait de nourrir", nbre_theorique, "personnes")

En théorie, la disponibilité alimentaire mondiale d'une journée permettrait de nourrir 8716243587.09 personnes


3. Même question pour la disponibilité alimentaire des produits végétaux.

Je filtre dans un premier temps sur l'origine végétale.

In [841]:
vegetal_kcal = dispo_alimentaire.iloc[:, 0:6]
vegetal_kcal = vegetal_kcal.loc[vegetal_kcal['Origine'] == 'vegetale', :]
vegetal_kcal[['Pays', 'Origine', 'Disponibilité alimentaire (Kcal/personne/jour)']]

,Pays,Origine,Disponibilité alimentaire (Kcal/personne/jour)
1,Afghanistan,vegetale,1.0
2,Afghanistan,vegetale,1.0
3,Afghanistan,vegetale,0.0
4,Afghanistan,vegetale,4.0
6,Afghanistan,vegetale,0.0
...,...,...,...
15595,Îles Salomon,vegetale,0.0
15596,Îles Salomon,vegetale,0.0
15597,Îles Salomon,vegetale,0.0
15603,Îles Salomon,vegetale,0.0


De nouveau je joins le fichier population et fait un regroupement par pays. Puis je calcule le disponibilité alimentaire végétale totale par pays.

In [845]:
vegetal_kcal = vegetal_kcal.groupby('Pays').agg(np.sum)
vegetal_kcal = pd.merge(vegetal_kcal, population_2017, how='inner', left_on='Pays', right_on='Pays')
vegetal_kcal[['Pays', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Année', 'Ptot (M)']]

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Année,Ptot (M)
0,Afghanistan,1871.0,2017,36296113.0
1,Afrique du Sud,2533.0,2017,57009756.0
2,Albanie,2203.0,2017,2884169.0
3,Algérie,2915.0,2017,41389189.0
4,Allemagne,2461.0,2017,82658409.0
...,...,...,...,...
167,Émirats arabes unis,2718.0,2017,9487203.0
168,Équateur,1732.0,2017,16785361.0
169,États-Unis d'Amérique,2698.0,2017,325084756.0
170,Éthiopie,2005.0,2017,106399924.0


In [847]:
vegetal_kcal['Dispo vege par pays'] = vegetal_kcal['Disponibilité alimentaire (Kcal/personne/jour)'] * pays_kcal['Ptot (M)']
vegetal_kcal[['Pays', 'Disponibilité alimentaire (Kcal/personne/jour)', 'Année', 'Ptot (M)', 'Dispo vege par pays']]

,Pays,Disponibilité alimentaire (Kcal/personne/jour),Année,Ptot (M),Dispo vege par pays
0,Afghanistan,1871.0,2017,36296113.0,6.791003e+10
1,Afrique du Sud,2533.0,2017,57009756.0,1.444057e+11
2,Albanie,2203.0,2017,2884169.0,6.353824e+09
3,Algérie,2915.0,2017,41389189.0,1.206495e+11
4,Allemagne,2461.0,2017,82658409.0,2.034223e+11
...,...,...,...,...,...
167,Émirats arabes unis,2718.0,2017,9487203.0,2.578622e+10
168,Équateur,1732.0,2017,16785361.0,2.907225e+10
169,États-Unis d'Amérique,2698.0,2017,325084756.0,8.770787e+11
170,Éthiopie,2005.0,2017,106399924.0,2.133318e+11


In [850]:
nbre_theorique_vege = round((vegetal_kcal['Dispo vege par pays'].sum())/2400, 2)
print("En théorie, la disponibilité alimentaire végétale mondiale d'une journée permettrait de nourrir", nbre_theorique_vege, "personnes")

En théorie, la disponibilité alimentaire végétale mondiale d'une journée permettrait de nourrir 7191985082.26 personnes


4. L’utilisation de la disponibilité intérieure, en particulier la part qui est attribuée à l’alimentation animale, celle qui est perdue et celle qui est concrètement utilisée pour l'alimentation humaine. 

Pour calculer ces proportions je vais utiliser la formule suivante (issue du fichier sur la disponibilité alimentaire) : 
Nourriture + Aliments pour animaux + Perte + Semence + Traitement + Autres utilisations =
Disponibilité intérieure =
Production + Importation + Variation du stock - Exportation


In [852]:
part_alim_animale = round ((dispo_alimentaire['Aliments pour animaux'].sum() / dispo_alimentaire['Disponibilité intérieure'].sum()) * 100, 2)
print("La part attribuée à l'alimentation animale est de", part_alim_animale, "%")

La part attribuée à l'alimentation animale est de 13.24 %


In [855]:
part_pertes = round ((dispo_alimentaire['Pertes'].sum() / dispo_alimentaire['Disponibilité intérieure'].sum()) * 100, 2)
print("La part attribuée aux pertes est de", part_pertes, "%")

La part attribuée aux pertes est de 4.61 %


In [856]:
part_alim_humaine = round ((dispo_alimentaire['Nourriture'].sum() / dispo_alimentaire['Disponibilité intérieure'].sum()) * 100, 2)
print("La part concrètement utilisée pour l'alimentation humaine est de", part_alim_humaine, "%")

La part concrètement utilisée pour l'alimentation humaine est de 49.51 %


## Partie 3 - Questions de Mélanie

1. Nous donner les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017

In [860]:
proportion_par_pays = proportion
proportion_par_pays['Proportion personnes sous-alimentées'] = round((proportion_par_pays['Personnes en sous alimentation (M)'] / proportion_par_pays['Ptot (M)']) * 100, 2)
proportion_par_pays[['Pays', 'Proportion personnes sous-alimentées']].sort_values('Proportion personnes sous-alimentées', ascending = False).head(10)


,Pays,Proportion personnes sous-alimentées
472,Haïti,48.26
946,République populaire démocratique de Corée,47.19
652,Madagascar,41.06
622,Libéria,38.28
604,Lesotho,38.25
1102,Tchad,37.96
970,Rwanda,35.06
730,Mozambique,32.81
1120,Timor-Leste,32.17
4,Afghanistan,28.93


2. Les pays qui ont le plus bénéficié d’aide depuis 2013

In [803]:
aide_totale = aide_alimentaire[['Quantité (t)', 'Pays']].groupby('Pays').sum()
aide_totale.sort_values('Quantité (t)', ascending=False).head(10)

,Quantité (t)
Pays,
République arabe syrienne,1858943
Éthiopie,1381294
Yémen,1206484
Soudan du Sud,695248
Soudan,669784
Kenya,552836
Bangladesh,348188
Somalie,292678
République démocratique du Congo,288502


3. Les pays ayant le plus/le moins de disponibilité/habitant

Je calcule d'abord la somme par pays de la disponibilité alimentaire par personne puis je classe pour obternir les 10 premiers et les 10 derniers.

In [713]:
dispo_pays = dispo_alimentaire.sort_values('Disponibilité alimentaire (Kcal/personne/jour)').groupby('Pays', as_index=False)['Disponibilité alimentaire (Kcal/personne/jour)'].sum()
dispo_pays

,Pays,Disponibilité alimentaire (Kcal/personne/jour)
0,Afghanistan,2087.0
1,Afrique du Sud,3020.0
2,Albanie,3188.0
3,Algérie,3293.0
4,Allemagne,3503.0
...,...,...
169,Émirats arabes unis,3275.0
170,Équateur,2346.0
171,États-Unis d'Amérique,3682.0
172,Éthiopie,2129.0


In [715]:
dispo_pays.sort_values('Disponibilité alimentaire (Kcal/personne/jour)', ascending=False).head(10)

,Pays,Disponibilité alimentaire (Kcal/personne/jour)
11,Autriche,3770.0
16,Belgique,3737.0
159,Turquie,3708.0
171,États-Unis d'Amérique,3682.0
74,Israël,3610.0
72,Irlande,3602.0
75,Italie,3578.0
89,Luxembourg,3540.0
168,Égypte,3518.0
4,Allemagne,3503.0


In [716]:
dispo_pays.sort_values('Disponibilité alimentaire (Kcal/personne/jour)', ascending=True).head(10)

,Pays,Disponibilité alimentaire (Kcal/personne/jour)
128,République centrafricaine,1879.0
166,Zambie,1924.0
91,Madagascar,2056.0
0,Afghanistan,2087.0
65,Haïti,2089.0
133,République populaire démocratique de Corée,2093.0
151,Tchad,2109.0
167,Zimbabwe,2113.0
114,Ouganda,2126.0
154,Timor-Leste,2129.0


# Partie 3 : Questions de Julien

1. Lors de mes recherches, j’ai découvert des chiffres intéressants sur l’utilisation des céréales,
notamment la répartition entre l’alimentation humaine (colonne Nourriture) et l’alimentation
pour animaux. La liste des céréales peut être trouvée en allant fouiller un peu dans l’onglet
Bilans alimentaires, sur le lien juste ci-dessus. Peux-tu approfondir ce point ?

Je selectionne à partir de la liste des végétaux, les différentes céréles présentes dans le fichier. Je les mets dans une liste pour pouvoir trier seulement sur les céréales et faire mon calcul de répartition globale entre nourriture et consommation animale.

In [861]:
produits = dispo_alimentaire.loc[dispo_alimentaire['Origine'] == 'vegetale', :]
produits.head()
produits['Produit'].unique()

array(['Agrumes, Autres', 'Aliments pour enfants', 'Ananas', 'Bananes',
       'Bière', 'Blé', 'Boissons Alcooliques', 'Café',
       'Coco (Incl Coprah)', 'Céréales, Autres', 'Dattes',
       'Edulcorants Autres', 'Feve de Cacao', 'Fruits, Autres',
       'Graines de coton', 'Graines de tournesol',
       'Huil Plantes Oleif Autr', 'Huile Graines de Coton',
       "Huile d'Arachide", "Huile d'Olive", 'Huile de Colza&Moutarde',
       'Huile de Palme', 'Huile de Soja', 'Huile de Sésame',
       'Huile de Tournesol', 'Légumes, Autres', 'Légumineuses Autres',
       'Maïs', 'Miel', 'Millet', 'Miscellanees', 'Noix', 'Olives',
       'Oranges, Mandarines', 'Orge', 'Plantes Oleiferes, Autre',
       'Poivre', 'Pommes', 'Pommes de Terre', 'Raisin',
       'Riz (Eq Blanchi)', 'Sucre Eq Brut', 'Sucre, betterave',
       'Sucre, canne', 'Sésame', 'Thé', 'Tomates', 'Vin',
       'Épices, Autres', 'Alcool, non Comestible',
       'Arachides Decortiquees', 'Avoine', 'Bananes plantains',
       'Bo

In [862]:
liste_céréales = ['Blé', 'Céréales, Autres', 'Maïs', 'Millet', 'Orge', 'Riz (Eq Blanchi)', 'Seigle', 'Sorgho', 'Soja']
tab_cereales = dispo_alimentaire[dispo_alimentaire.Produit.isin(liste_céréales)]
tab_cereales.head()

,Pays,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
7,Afghanistan,Blé,vegetale,0.0,0.0,1369.0,160.23,4.69,36.91,5992.0,0.0,1173.0,4895.0,775.0,5169.0,322.0,0.0,-350.0
12,Afghanistan,"Céréales, Autres",vegetale,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,Afghanistan,Maïs,vegetale,200.0,0.0,21.0,2.50,0.30,0.56,313.0,0.0,1.0,76.0,31.0,312.0,5.0,0.0,0.0
34,Afghanistan,Millet,vegetale,0.0,0.0,3.0,0.40,0.02,0.08,13.0,0.0,0.0,12.0,1.0,13.0,0.0,0.0,0.0
40,Afghanistan,Orge,vegetale,360.0,0.0,26.0,2.92,0.24,0.79,524.0,0.0,10.0,89.0,52.0,514.0,22.0,0.0,0.0


In [863]:
tab_cereales = tab_cereales.groupby('Produit').sum()
tab_cereales

,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
Produit,,,,,,,,,,,,,,,
Blé,129668.0,22424.0,96132.0,12339.51,548.76,2809.21,679498.0,198380.0,185377.0,457824.0,27530.0,708443.0,34283.0,7877.0,-15948.0
"Céréales, Autres",19035.0,1388.0,1483.0,193.74,5.46,40.52,27485.0,2540.0,2882.0,5324.0,785.0,28412.0,1203.0,160.0,-1267.0
Maïs,546116.0,189386.0,30903.0,3690.37,271.50,771.12,955799.0,128787.0,122691.0,125184.0,40701.0,1017029.0,6837.0,47648.0,-55136.0
Millet,3306.0,196.0,3966.0,513.09,37.27,96.11,29911.0,276.0,343.0,23040.0,2276.0,28909.0,682.0,403.0,933.0
Orge,92658.0,654.0,1814.0,263.60,7.35,52.28,140439.0,41585.0,38689.0,6794.0,4628.0,142462.0,8804.0,26910.0,876.0
Riz (Eq Blanchi),33594.0,18464.0,53024.0,5384.46,123.95,1027.25,475656.0,41445.0,37618.0,377286.0,27378.0,495447.0,12194.0,6740.0,-15958.0
Seigle,8099.0,20.0,1967.0,263.36,8.62,51.56,16567.0,2372.0,2251.0,5502.0,562.0,16711.0,1262.0,1120.0,-25.0
Soja,17476.0,638.0,1086.0,127.12,47.72,109.71,267445.0,106912.0,102082.0,10648.0,3970.0,278361.0,7401.0,227314.0,-6082.0
Sorgho,24808.0,2049.0,4952.0,614.02,48.34,146.16,58237.0,6388.0,6418.0,24153.0,2746.0,62116.0,788.0,3695.0,-3913.0


In [864]:
animaux = round((tab_cereales['Aliments pour animaux'].sum()/tab_cereales['Disponibilité intérieure'].sum()) * 100, 2)
print("Pourcentage d'utilisation de céréales pour l'alimentation animale :", animaux, "%")

Pourcentage d'utilisation de céréales pour l'alimentation animale : 33.0 %


In [865]:
humain = round((tab_cereales['Nourriture'].sum()/tab_cereales['Disponibilité intérieure'].sum()) * 100, 2)
print("Pourcentage d'utilisation de céréales pour l'alimentation humaine :", humain, "%")

Pourcentage d'utilisation de céréales pour l'alimentation humaine : 39.07 %


Il y a un tiers des céréales qui sont utilisés pour nourrir les animaux qui seront ensuite manger à leur tour par des humains. Peut être serait-il mieux de réserver une plus grande part des céréales à la nourriture humaine directement.

2. Enfin, je me souviens d’un exemple étonnant d’utilisation du manioc par la Thaïlande aux
égards de la proportion de personnes en sous-nutrition. De mémoire ça concernait
l’exportation par rapport à la production… Peux-tu jeter un coup d’œil et nous faire un retour
?

In [783]:
manioc = dispo_alimentaire.loc[(dispo_alimentaire['Produit'] == 'Manioc') & (dispo_alimentaire['Pays'] == 'Thaïlande')]
manioc

,Pays,Produit,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
13809,Thaïlande,Manioc,vegetale,1800.0,2081.0,40.0,13.0,0.05,0.14,6264.0,25214.0,1250.0,871.0,1511.0,30228.0,NaN,0.0,0.0


Utilisation de la methode np.asarray pour passer le vecteur manioc_exporte en float.

In [816]:
manioc_exporte = round((manioc['Exportations - Quantité'] / manioc['Production']) * 100, 2)
manioc_exporte = np.asarray(manioc_exporte, dtype=float)[0]
print("Proportion d'exportation du manioc par la Thaïlande :", manioc_exporte, "%")

Proportion d'exportation du manioc par la Thaïlande : 83.41 %


In [792]:
proportion_thailande = proportion_par_pays.loc[proportion_par_pays['Pays'] == 'Thaïlande', :]
proportion_thailande

,Pays,Année,Personnes en sous alimentation (M),Ptot (M),Total par pays
1114,Thaïlande,2017,6200000.0,69209810,8.96


En faisant un simple produit en croix on se rend compte que si la Thaîlande gardait ses exportations de manioc elle pourrait multiplier par 30 l'apport calorique du manioc par personne par jour. Elle ferait baisser ainsi son taux de personnes en sous nutrition.